In [1]:
import pandas as pd
from googleapiclient.discovery import build
import googlemaps
import time
import os
import csv
from dotenv import load_dotenv

In [2]:
load_dotenv()
# Google API credentials
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
GOOGLE_CSE_ID = os.getenv('GOOGLE_CSE_ID')

In [3]:
# Initialize Google Maps client
gmaps = googlemaps.Client(key=GOOGLE_API_KEY)


In [4]:
# Filenames for state management
STATE_FILE = 'script_state.csv'
ERROR_FILE = 'errors.csv'

In [5]:
def create_search_query(row):
    """
    Create a search query from company information.
    Filter out any None or NaN values and join with spaces.
    """
    components = [
        row['Company name'],
        row['Country'],
        row['City'],
        row['Province'],
        row['Site address']
    ]
    query = ' '.join(str(item) for item in components if pd.notna(item))
    return query

In [35]:
def extract_website_url(search_result):
    """
    Extract the most likely website URL from the top few search results.
    Skip any aggregator or known social media domains.
    """
    if not search_result.get('items'):
        print("No search results found.")
        return None
    
    excluded_domains = [
        'facebook.com', 'linkedin.com', 'twitter.com',
        'instagram.com', 'youtube.com'
    ]
    print("Checking the first five results for a legitimate domain...")
    # Check the first five results for a legit domain
    for index, item in enumerate(search_result['items'][:5]):
        url = item['link'].lower()
        print(f"Checking result {index + 1}: {url}")
        if not any(domain in url for domain in excluded_domains):
            print(f"Valid website found: {item['link']}")
            return item['link']
    
    print("No valid websites found in the top results.")
    return None

In [37]:
def update_location_info(row):
    """
    Update missing location information using Google Places API.
    Check multiple Places results to find the most relevant.
    """
    try:
        # Construct a location-based search query (company + city + country).
        search_query = row['Company name']
        print(f"Constructing search query for: {search_query}")
        if pd.notna(row['City']):
            search_query += f", {row['City']}"
            print(f"Added city to search query: {row['City']}")
        if pd.notna(row['Country']):
            search_query += f", {row['Country']}"
            print(f"Added country to search query: {row['Country']}")

        print("Sending request to Google Places API...")
        places_result = gmaps.places(search_query)

        if places_result.get('results'):
            print("Results received from Google Places API.")
            # Let's pick the best place if it exists, or default to the first.
            # You could refine this further by checking user_ratings_total, 
            # proximity, or other heuristics.
            
            # Sort results by rating or just pick top if rating is missing
            sorted_results = sorted(
                places_result['results'], 
                key=lambda x: x.get('rating', 0), 
                reverse=True
            )
            best_place = sorted_results[0]
            print(f"Best place found: {best_place['name']} with rating: {best_place.get('rating', 'N/A')}")
            
            # Fetch place details
            print(f"Fetching details for place ID: {best_place['place_id']}")
            place_details = gmaps.place(best_place['place_id'])['result']
            
            address_components = place_details.get('address_components', [])
            
            location_info = {
                'Country': None,
                'City': None,
                'Province': None,
                'Site address': place_details.get('formatted_address')
            }
            
            for component in address_components:
                types = component['types']
                if 'country' in types:
                    location_info['Country'] = component['long_name']
                    print(f"Country found: {component['long_name']}")
                elif 'locality' in types:
                    location_info['City'] = component['long_name']
                    print(f"City found: {component['long_name']}")
                elif 'administrative_area_level_1' in types:
                    location_info['Province'] = component['long_name']
                    print(f"Province found: {component['long_name']}")
            
            return location_info
            
    except Exception as e:
        print(f"Error updating location info: {str(e)}")
    
    return None

In [39]:
def save_state(df, filename=STATE_FILE):
    """
    Save the current state of the data to a CSV so we can resume later.
    """
    print(f"Saving current state to {filename}...")
    df.to_csv(filename, index=False)
    print("State saved successfully.")

In [40]:
def load_state(filename=STATE_FILE):
    """
    Load a partial state file if it exists (i.e., resuming from a previous run).
    """
    print(f"Checking for existing state file: {filename}")
    if os.path.exists(filename):
        print("State file found. Loading...")
        return pd.read_csv(filename)
    print("No state file found. Starting fresh.")
    return None

In [41]:
def log_error(idx, message, row, filename=ERROR_FILE):
    """
    Log an error row to a separate CSV for debugging or re-processing later.
    """
    print(f"Logging error for index {idx}: {message}")
    with open(filename, 'a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([idx, message, dict(row)])
    print(f"Error logged for index {idx} successfully.")

In [51]:
def find_missing_info(df):
    """Find websites and update missing location information with error logging."""
    # Try to resume from a partial state if available
    resumed_df = load_state()
    if resumed_df is not None:
        print("Resuming from saved state...")
        df = resumed_df
    
    # Create Google Search API service
    service = build('customsearch', 'v1', developerKey=GOOGLE_API_KEY)
    
    # Create a copy of rows that still have missing info
    missing_info = df[
        df['Company website'].isna() |
        df['Country'].isna() |
        df['City'].isna() |
        df['Province'].isna() |
        df['Site address'].isna()
    ].copy()
    
    # Counter for daily quota usage
    requests_count = 0
    daily_quota = 100  # Or whatever your free tier limit is
    total_missing = len(missing_info)
    print(f"Total entries with missing information: {total_missing}")

    for idx, row in missing_info.iterrows():
        print(f"Processing row {idx + 1}/{total_missing}...")
        
        # Check if we still need to fill location or website
        location_missing = (
            pd.isna(row['Country']) or 
            pd.isna(row['City']) or 
            pd.isna(row['Province']) or 
            pd.isna(row['Site address'])
        )
        website_missing = pd.isna(row['Company website'])
        
        try:
            # Update location info
            if location_missing:
                location_info = update_location_info(row)
                if location_info:
                    for field, value in location_info.items():
                        # Update only if we're missing that piece
                        if pd.isna(row[field]) and value:
                            if pd.isna(df.at[idx, field]) and value is not None:
                                # Explicitly convert column to string dtype before setting value
                                if df[field].dtype != 'object':
                                    df[field] = df[field].astype('object')
                                df.at[idx, field] = str(value)
                print(f"Updated location info for row {idx + 1}.")
                # Sleep to be a bit polite to the API
                time.sleep(2)
            
            # Update website
            if website_missing:
                query = create_search_query(df.loc[idx])
                
                result = service.cse().list(
                    q=query,
                    cx=GOOGLE_CSE_ID,
                    num=5  # get up to 5 results
                ).execute()
                
                website_url = extract_website_url(result)
                if website_url:
                    # Convert column to string dtype before setting value
                    if df['Company website'].dtype != 'object':
                        df['Company website'] = df['Company website'].astype('object')
                    df.at[idx, 'Company website'] = str(website_url)
                    print(f"Updated website for row {idx + 1}.")
                
                requests_count += 1
                if requests_count >= daily_quota:
                    # Instead of sleeping 24 hours, let's store state & exit
                    print(f"Reached daily quota of {daily_quota} requests. Stopping...")
                    save_state(df)
                    return df
                else:
                    # Minimal rate limit
                    time.sleep(1)
            
        except Exception as e:
            print(f"Error processing row {idx}: {str(e)}")
            log_error(idx, str(e), df.loc[idx])
            continue
    
    return df

In [49]:

def main():
    print("Loading the CSV file...")
    df = pd.read_csv('missing_websites.csv')
    print("CSV file loaded successfully. Finding and updating missing information...")
    
    updated_df = find_missing_info(df)
    print("Missing information updated successfully. Saving final results...")

    output_file = 'companies_info_updated.csv'
    if os.path.exists(output_file):
        timestamp = int(time.time())
        output_file = f'companies_info_updated_{timestamp}.csv'
    
    updated_df.to_csv(output_file, index=False)
    print("Final results saved to 'companies_info_updated.csv'.")
    # Print summary
    print("\nSummary of updates:")
    for column in ['Company website', 'Country', 'City', 'Province', 'Site address']:
        before = len(df[df[column].isna()])
        after = len(updated_df[updated_df[column].isna()])
        print(f"{column}:")
        print(f"  Missing before: {before}")
        print(f"  Missing after: {after}")
        print(f"  Found: {before - after}")
        print()

    # Clear out any saved state at successful completion
    if os.path.exists(STATE_FILE):
        os.remove(STATE_FILE)
        print("Removed temporary state file after successful run.")

In [52]:
main()

Loading the CSV file...
CSV file loaded successfully. Finding and updating missing information...
Checking for existing state file: script_state.csv
No state file found. Starting fresh.
Total entries with missing information: 492
Processing row 1/492...
Constructing search query for: Proteins Australia Pty Ltd
Added country to search query: Australia
Sending request to Google Places API...
Results received from Google Places API.
Best place found: Australian Plant Proteins with rating: 5
Fetching details for place ID: ChIJs51ZbWBbzmoRvJFrOcsIBYo
City found: Horsham
Province found: Victoria
Country found: Australia
Updated location info for row 1.
Checking the first five results for a legitimate domain...
Checking result 1: https://approteins.com.au/
Valid website found: https://approteins.com.au/
Updated website for row 1.
Processing row 2/492...
Constructing search query for: Mundo Branco S. A.
Added country to search query: Argentina
Sending request to Google Places API...
Results re